In [1]:
import sys
import collections 
import laf
from laf.fabric import LafFabric
from etcbc.preprocess import prepare

fabric = LafFabric(verbose='NORMAL')

  0.00s This is LAF-Fabric 4.5.21
API reference: http://laf-fabric.readthedocs.org/en/latest/texts/API-reference.html
Feature doc: https://shebanq.ancient-data.org/static/docs/featuredoc/texts/welcome.html



In [2]:
fabric.load('etcbc4b', '--', 'monad_search',
{
    "xmlids" : {"node": False, "edge" : False},
    "features" : ("oid otype monads chapter verse book g_word_utf8 trailer_utf8",""),
    "prepare" : prepare
}
           )
exec(fabric.localnames.format(var='fabric'))

  0.00s LOADING API: please wait ... 
  0.00s USING main  DATA COMPILED AT: 2015-11-02T15-08-56
  4.61s LOGFILE=/Users/Cody/laf-fabric-output/etcbc4b/monad_search/__log__monad_search.txt
  4.61s INFO: LOADING PREPARED data: please wait ... 
  4.61s prep prep: G.node_sort
  4.73s prep prep: G.node_sort_inv
  5.30s prep prep: L.node_up
  9.13s prep prep: L.node_down
    15s prep prep: V.verses
    15s prep prep: V.books_la
    15s ETCBC reference: http://laf-fabric.readthedocs.org/en/latest/texts/ETCBC-reference.html
    18s INFO: LOADED PREPARED data
    18s INFO: DATA LOADED FROM SOURCE etcbc4b AND ANNOX lexicon FOR TASK monad_search AT 2016-03-10T20-22-59


In [3]:
corpus = "Canticum"
cur_book = None
nodes = []
for n in NN():
    otype = F.otype.v(n)
    if cur_book==corpus:
        nodes.append(n)
    elif otype == "book":
        cur_book = F.book.v(n)
msg("{} nodes added".format(len(nodes)))

    15s 218647 nodes added


In [8]:
filename = 'ROBchapter_8'


import csv
sticho_dict = collections.OrderedDict([])
monad_list = []
type_codes = {"word":"W", "phrase_atom":"Pa", "clause_atom":"Ca", "sentence_atom":"Sa", "subphrase":"Sp", 
              "phrase":"P","clause":"C","sentence":"S","half_verse":"Hv", "verse": "V"}

with open('{}.csv'.format(filename), 'r') as csvfile2:
    readit = csv.reader(csvfile2)
    for line in readit:
        sticho_dict[int(line[0])] = [int(line[1]), line[2], line[3], line[4]]

for key in sticho_dict:
    monad_list.append([key,sticho_dict[key][0]])
    

sysct = 0
for pair in monad_list:
    lookup = '{}-{}'.format(pair[0],pair[1])
    monads = tuple(int(x) for x in lookup.split('-'))
    monad_list = [str(x) for x in range(monads[0], monads[1] +1)]
    
    #looks up every word between m_1 and m_2
    monad_set = set(monad_list)
    
    lookedup = []
    words = {}
    

    for node in nodes:
        mon = F.monads.v(node)
        if mon == lookup:
            lookedup.append(node)
        if mon in monad_set:
            words[mon] = node
    
    otype_code = ""
    
    if lookedup != []:
        dotct = len(lookedup)
        for n in lookedup:
            otype = F.otype.v(n)
            if otype in type_codes:
                dotct -= 1
                if dotct > 0:
                    otype_code += type_codes[otype]+"."
                else:
                    otype_code += type_codes[otype]
    
    else:
        otype_code += "N"
    
    sticho_dict[pair[0]].append(otype_code)
    sysct += 1
    msg("( {}/{} )  ({})".format(sysct,len(sticho_dict),otype_code))

 1m 04s ( 1/52 )  (C.Ca)
 1m 05s ( 2/52 )  (C.Ca)
 1m 05s ( 3/52 )  (N)
 1m 05s ( 4/52 )  (S.Sa.C.Ca)
 1m 05s ( 5/52 )  (N)
 1m 05s ( 6/52 )  (N)
 1m 06s ( 7/52 )  (N)
 1m 06s ( 8/52 )  (Pa)
 1m 06s ( 9/52 )  (S.Sa.C.Ca)
 1m 06s ( 10/52 )  (S.Sa.C.Ca)
 1m 06s ( 11/52 )  (C.Ca)
 1m 07s ( 12/52 )  (C.Ca.P.Pa)
 1m 07s ( 13/52 )  (N)
 1m 07s ( 14/52 )  (C.Ca)
 1m 07s ( 15/52 )  (C.Ca)
 1m 07s ( 16/52 )  (C.Ca)
 1m 08s ( 17/52 )  (S.Sa.C.Ca)
 1m 08s ( 18/52 )  (S.Sa.C.Ca)
 1m 08s ( 19/52 )  (S.Sa)
 1m 08s ( 20/52 )  (C.Ca)
 1m 08s ( 21/52 )  (C.Ca)
 1m 09s ( 22/52 )  (S.Sa.C.Ca)
 1m 09s ( 23/52 )  (S.Sa.C.Ca)
 1m 09s ( 24/52 )  (N)
 1m 09s ( 25/52 )  (N)
 1m 09s ( 26/52 )  (Hv)
 1m 09s ( 27/52 )  (Hv)
 1m 10s ( 28/52 )  (S.Sa.C.Ca)
 1m 10s ( 29/52 )  (S.Sa.C.Ca)
 1m 10s ( 30/52 )  (N)
 1m 10s ( 31/52 )  (N)
 1m 11s ( 32/52 )  (S.Sa.C.Ca)
 1m 11s ( 33/52 )  (S.Sa.C.Ca)
 1m 11s ( 34/52 )  (S.Sa.C.Ca)
 1m 11s ( 35/52 )  (S.Sa.C.Ca)
 1m 11s ( 36/52 )  (S.Sa.C.Ca)
 1m 12s ( 37/52 )  (S.Sa.C.Ca)


In [9]:
with open('{}_coded.csv'.format(filename), 'w') as csvfile:
    writer = csv.writer(csvfile)
    sysct = 0
    for key in sticho_dict:
        writer.writerow([sticho_dict[key][1]]+[key]+[sticho_dict[key][0]]+[sticho_dict[key][2]]+[sticho_dict[key][3]]+[sticho_dict[key][4]])
msg("job finished! file saved.")

 1m 23s job finished! file saved.


In [10]:
total = 0
totalN = 0
types = []
accents = []
for key in sticho_dict:
    total += 1
    if sticho_dict[key][4] == "N":
        types.append(sticho_dict[key][2])
        accents.append(sticho_dict[key][3])
        totalN += 1

print ("total stichoi = " + (str(total)))
print ("total none = " + (str(totalN)))
print (str(round(float(totalN / total)*100,2)) + "%")
print (types)

total stichoi = 52
total none = 13
25.0%
['c', 'a', 'b', 'c', 'c', 'e', 'f', 'c', 'd', 'c', 'b', 'c', 'c']
